In [1]:
import pandas as pd
import nltk
from nltk.corpus import *
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
ds_train=pd.read_csv('/content/drive/MyDrive/IMDB_dataset/Train.csv')
ds_test=pd.read_csv('/content/drive/MyDrive/IMDB_dataset/Test.csv')
ds_valid=pd.read_csv('/content/drive/MyDrive/IMDB_dataset/Valid.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/IMDB_dataset/Train.csv'

In [5]:
ds_train.info()

NameError: name 'ds_train' is not defined

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
def preprocessing(text):
  text=text.lower()
  text = re.sub(r'[^a-z\s]', '', text)
  tokenized_text=word_tokenize(text)
  stopwords_list=nltk.corpus.stopwords.words(fileids='english')
  no_stopwords=[word for word in tokenized_text if word not in stopwords_list]
  lemmatizer=WordNetLemmatizer()
  lemmatized_words=[lemmatizer.lemmatize(word) for word in no_stopwords]
  preprocessed_text=' '.join(lemmatized_words)
  return preprocessed_text

In [ ]:
ds_train['text']=ds_train['text'].apply(preprocessing)
ds_test['text']=ds_test['text'].apply(preprocessing)
ds_valid['text']=ds_valid['text'].apply(preprocessing)

In [6]:
tokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=10000,oov_token='<oov>')
tokenizer.fit_on_texts(ds_train['text'])

NameError: name 'ds_train' is not defined

In [ ]:
train_seq=tokenizer.texts_to_sequences(ds_train['text'])
test_seq=tokenizer.texts_to_sequences(ds_test['text'])
valid_seq=tokenizer.texts_to_sequences(ds_valid['text'])

In [ ]:
max_len=100
train_padded=tf.keras.preprocessing.sequence.pad_sequences(train_seq,maxlen=max_len)
test_padded=tf.keras.preprocessing.sequence.pad_sequences(test_seq,maxlen=max_len)
valid_padded=tf.keras.preprocessing.sequence.pad_sequences(valid_seq,maxlen=max_len)

In [ ]:
AUTOTUNE=tf.data.experimental.AUTOTUNE
training=tf.data.Dataset.from_tensor_slices((train_padded,ds_train['label'])).batch(32).prefetch(AUTOTUNE)
testing=tf.data.Dataset.from_tensor_slices((test_padded,ds_test['label'])).batch(32).prefetch(AUTOTUNE)
validation=tf.data.Dataset.from_tensor_slices((valid_padded,ds_valid['label'])).batch(32).prefetch(AUTOTUNE)


In [ ]:
model=keras.Sequential([
    layers.Embedding(10000+1,300),
    layers.GlobalAveragePooling1D(),
    layers.Dense(60,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(3e-4),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(training,validation_data=validation,epochs=10,verbose=1)